In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import moviepy
import moviepy.editor as mp
import time
from IPython.display import clear_output
from tqdm import tqdm
from ultralytics import YOLO
import torch
import av
import numpy as np
import pandas as pd
from PIL import Image
import moviepy
import moviepy.editor as mp
from IPython.display import clear_output
import tempfile
import seaborn as sns
from transformers import DetrImageProcessor, DetrForObjectDetection
import cv2
from datetime import datetime


# from IPython.display import Video

In [2]:
# Define a function to determine the state based on conditions
def determine_state(row):
    if row['Healthy']:
        return 'Healthy'
    elif row['Diarrhé'] and row['Pneumonie']:
        return 'Diarrhé_Pneumonie'
    elif row['Diarrhé']:
        return 'Diarrhé'
    elif row['Pneumonie']:
        return 'Pneumonie'
    else:
        return 'Unknown'

def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"


# Function to sample uniformly
def uniform_sample(df, sample_size, groupby_cols):
    # Calculate the number of rows to sample per group
    num_groups = df.groupby(groupby_cols).ngroups
    sample_per_group = max(sample_size // num_groups, 1)
    
    # Sample uniformly within each group
    sampled_df = df.groupby(groupby_cols, group_keys=False).apply(lambda x: x.sample(min(len(x), sample_per_group)))
    
    # If we sampled less than the desired sample_size due to group sizes, sample more from the remaining dataframe
    # if len(sampled_df) < sample_size:
    #     remaining_sample_size = sample_size - len(sampled_df)
    #     remaining_df = df[~df.index.isin(sampled_df.index)]
    #     additional_samples = remaining_df.sample(min(len(remaining_df), remaining_sample_size))
    #     sampled_df = pd.concat([sampled_df, additional_samples])
    
    return sampled_df

# Function to sample uniformly with specified values
def uniform_sample_with_values(df, sample_size, groupby_cols, filter_values = None):
    copy = df.copy(deep=True)
    # Filter dataframe based on specified values for each column
    if filter_values is not None:
        for col, values in filter_values.items():
            copy = copy[copy[col].isin(values)]
    
    # Calculate the number of rows to sample per group
    num_groups = copy.groupby(groupby_cols).ngroups
    sample_per_group = max(sample_size // num_groups, 1)
    
    # Sample uniformly within each group
    sampled_df = copy.groupby(groupby_cols, group_keys=False).apply(lambda x: x.sample(min(len(x), sample_per_group)))
    
    # If we sampled less than the desired sample_size due to group sizes, sample more from the remaining dataframe
    if len(sampled_df) < sample_size:
        remaining_sample_size = sample_size - len(sampled_df)
        remaining_df = copy[~copy.index.isin(sampled_df.index)]
        additional_samples = remaining_df.sample(min(len(remaining_df), remaining_sample_size))
        sampled_df = pd.concat([sampled_df, additional_samples])
    
    return sampled_df, copy[~copy.index.isin(sampled_df.index)]

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    if converted_len >= seg_len:
        end_idx = seg_len
    else:
        end_idx = np.random.randint(converted_len, seg_len)
    # start_idx = end_idx - converted_len
    # start_idx = clip_len
    start_idx = 0
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

def plot_results(pil_img, scores, boxes, labels, masks=None):
    plt.figure(figsize=(16,10))
    np_image = np.array(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if masks is None:
      masks = [None for _ in range(len(scores))]
        
    assert len(scores) == len(boxes) == len(labels) == len(masks)
    
    for s, (xmin, ymin, xmax, ymax), l, mask, c in zip(scores, boxes.tolist(), labels, masks, colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{l}: {s:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

        if mask is None:
          continue
        np_image = apply_mask(np_image, mask, c)

        padded_mask = np.zeros((mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
          verts = np.fliplr(verts) - 1
          p = Polygon(verts, facecolor="none", edgecolor=c)
          ax.add_patch(p)


    plt.imshow(np_image)
    plt.axis('off')
    plt.show()


def add_res(results, ax, color='green'):

    bboxes = results['boxes']
    labels = results['labels']
    scores = results['scores']

    colors = ['purple', 'yellow', 'red', 'green', 'orange', 'pink']
    
    for i, (b, ll, ss) in enumerate(zip(bboxes, labels, scores)):
        ax.add_patch(plt.Rectangle((b[0], b[1]), b[2] - b[0], b[3] - b[1], fill=False, color=colors[i], linewidth=3))
        cls_name = ll if isinstance(ll,str) else CLASSES[ll]
        text = f'{cls_name}: {ss:.2f}'
        print(text)
        ax.text(b[0], b[1], text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))


def plot_gallery(image_paths, n_cols=5):
    n_images = len(image_paths)
    n_rows = n_images // n_cols + int(n_images % n_cols > 0)
    
    plt.figure(figsize=(20, n_rows * 4))
    print(n_rows, n_cols)
    for i, image_path in enumerate(image_paths):
        img = Image.open(image_path)
        plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(img)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()   

def plot_bbox_gallery(image_df, n_cols=5):
    n_images = image_df.shape[0]
    n_rows = n_images // n_cols + int(n_images % n_cols > 0)
    
    plt.figure(figsize=(20, n_rows * 4))
    for i, row in image_df.iterrows():
        img = Image.open(row["path"])
        # plt.subplot(n_rows, n_cols, i + 1)
        ax = plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(img)
        # ax = plt.gca()
        score = row["conf"]
        box_x = row["box_x"]
        box_y = row["box_y"]
        box_width = row["box_width"]
        box_height = row["box_height"]
        box_x = box_x - (box_width / 2)
        box_y = box_y - (box_height / 2)
        rect = plt.Rectangle((box_x, box_y), box_width, box_height, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(box_x, box_y - 10, f' {score:.2f}', color='red', fontsize=12, backgroundcolor='white')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()   

In [30]:
def extract_medias(aptms, mp4_files, root_dir, frame_sample_rate = 4, num_images = 16, bbox_conf = (.8, .5), delta = 20, step = 0, video_model = None,  image_model = None):
    
    size = len(aptms)
    video_records = []
    image_records = []
    indices_records = []
    frame = delta - step
    for idx in tqdm(range(size), desc="Processing items"):
        one_row = aptms.iloc[idx]
        target = one_row['State']
        aptm_end_at = one_row["datetime"] +  pd.Timedelta(seconds=one_row["Duration"]) 
        video_end_at = aptm_end_at +  pd.Timedelta(seconds=delta) 
            
        cond = (
            (mp4_files["start_date"] == one_row.datetime.date())
            & (mp4_files["start_time"] <= aptm_end_at)
            & (mp4_files["end_time"] > aptm_end_at)
            & (mp4_files["station"] == one_row.parc)
        )

        if mp4_files[cond].shape[0] == 0:
            continue

        video_dir_path = root_dir + "/videos/" + target
        if not os.path.exists(video_dir_path):
            os.makedirs(video_dir_path)

        image_dir_path = root_dir + "/images/" + target
        if not os.path.exists(image_dir_path):
            os.makedirs(image_dir_path)

        for index, file in mp4_files[cond].iterrows():
            zero = (file["end_time"] - video_end_at).total_seconds()
            if zero >= 0:
                break
    
        video = mp.VideoFileClip(file["path"])
        
        start_at = max((aptm_end_at - file["start_time"]).total_seconds(), 0)
        end_at = max((video_end_at - file["start_time"]).total_seconds(), 0)

        print(start_at, end_at, file["start_time"], one_row["datetime"], aptm_end_at, video_end_at, file["path"], one_row["calfNumber"])
        
        # if end_at <= start_at or (end_at - start_at) <= 1:
        if end_at <= start_at or (end_at - start_at) < delta - step:
            continue
            
        clip = video.subclip(start_at, end_at)
        nfaces = 0
        
        with tempfile.NamedTemporaryFile(suffix='.mp4') as tmpfile:
            clip.write_videofile(tmpfile.name, logger= None)
            container = av.open(tmpfile.name)

            video_stream = container.streams.video[0]

            # Get the duration in time base units
            duration_in_units = video_stream.duration
            
            # Get the time base
            time_base = video_stream.time_base
            
            # Calculate the duration in seconds
            duration_in_seconds = duration_in_units * time_base

            fps = int(container.streams.video[0].average_rate)
            seg_len = int(duration_in_seconds) * fps
            sample_rate = min(int(seg_len // num_images), frame_sample_rate)
            indices = sample_frame_indices(clip_len=num_images, frame_sample_rate=sample_rate, seg_len=seg_len)
            video_frames = read_video_pyav(container, indices)
            image_frames = [Image.fromarray(frame) for frame in video_frames]

        results = image_model.predict(image_frames, save=False, imgsz=640, conf=bbox_conf[0], max_det=1, show=False)
        result_records = []
        image_indices = []
        for frame_id, r in enumerate(results):
            if r.boxes.shape[0] == 0:
                continue
            else:
                conf = r.boxes.cpu().numpy().conf[0]
                x1, y1, width, height =  r.boxes.cpu().numpy().xywh[0]

            img = image_frames[frame_id]
            filename = f"image_{idx}_{frame_id}_{one_row['calfNumber']}_{one_row['parc']}_{one_row['datetime']}_{target}.png"
            img_save_path = os.path.join(image_dir_path, filename)
            img.save(img_save_path)
            image_indices.append(indices[frame_id])

            result_records.append({
                "image": filename,
                "path": img_save_path,
                "label": target,
                "calf": one_row["calfNumber"],
                "station": one_row["parc"],
                "before": one_row["datetime"],
                "from": file["path"],
                "conf": conf, 
                "box_x": x1,
                "box_y": y1,
                "box_width": width,
                "box_height": height,
            })

        # if len(result_records) == 0:
        if len(result_records) < 1:
            continue

        nfaces = len(result_records)
        
        image_records.extend(result_records)
        filename = f"clip_{idx}_{one_row['calfNumber']}_{one_row['parc']}_{one_row['datetime']}_{target}.mp4"
        temp_path = f"{video_dir_path}/{filename}"
        clip.write_videofile(temp_path, logger= None)
        
        indices_records.extend([{"fps": fps, "seg_len": seg_len, "sample_rate": sample_rate, "idx": idx, "from": temp_path} for idx in image_indices])
        
        video_records.append({
            "video": filename,
            "path": temp_path,
            "label": target,
            "duration": end_at - start_at,
            "calf": one_row["calfNumber"],
            "station": one_row["parc"],
            "before": one_row["datetime"],
            "from": file["path"],
            "nfaces": nfaces
        })
    
    # clear_output()
    return pd.DataFrame(video_records), pd.DataFrame(image_records), pd.DataFrame(indices_records)

In [4]:
ROOT_DIR = "/data/konrad/workspace"
root_dir = f"/data/data_calves/konrad"

yolo_id = 7
yolo_face = YOLO(ROOT_DIR + f"/models/best-face{yolo_id}.pt")
yolo_world = YOLO("yolov8s-world.pt")  
yolo_world.set_classes(["calf face"])

aptm_sampled = pd.read_csv(f"{root_dir}/usable_aptm_events.csv")
mp4_files = pd.read_csv(f"{root_dir}/mp4_files.csv")

aptm_sampled["datetime"] = pd.to_datetime(aptm_sampled["datetime"])
mp4_files["start_time"] = pd.to_datetime(mp4_files["start_time"])
mp4_files["end_time"] = pd.to_datetime(mp4_files["end_time"])
mp4_files["start_date"] = mp4_files["start_time"].dt.date

In [27]:
aptm_sampled.head()

,calfNumber,parc,day,Duration,cwd,datetime,State
0,6791,2,2022-02-18,478,6791 2022-02-18,2022-02-18 18:59:51,Diarrhé
1,6768,2,2022-02-18,136,6768 2022-02-18,2022-02-18 18:52:10,Diarrhé
2,6768,2,2022-02-18,128,6768 2022-02-18,2022-02-18 18:48:57,Diarrhé
3,6768,2,2022-02-18,203,6768 2022-02-18,2022-02-18 18:31:37,Diarrhé
4,6761,10,2022-02-18,1126,6761 2022-02-18,2022-02-18 18:30:56,Pneumonie


In [32]:
# Start time
start_time = time.time()

frame = 10
step = 0
frame_sample_rate = 15
num_images = 30
ibbox_conf = .80
vbbox_conf = .50
delta = frame + step
current_datetime = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
timestamp = f"y{yolo_id}_{current_datetime}"
root_dir = ROOT_DIR + f"/datasets/mixed_{frame}s_b{step}s_{timestamp}"
# root_dir = f"/data/data_calves/konrad/mixed_{frame}s_b{step}s"
# root_dir = f"/data/data_calves/konrad/mixed_{frame}s_b{step}s_{timestamp}"

# records = extract_medias(aptm_sampled, mp4_files, root_dir, delta = delta, step = step, frame_sample_rate = frame_sample_rate, num_images = num_images, video_model = yolo_world, image_model = yolo_face, bbox_conf = (ibbox_conf, vbbox_conf))
records = extract_medias(aptm_sampled.iloc[:10], mp4_files, root_dir, delta = delta, step = step, frame_sample_rate = frame_sample_rate, num_images = num_images, video_model = yolo_world, image_model = yolo_face, bbox_conf = (ibbox_conf, vbbox_conf))
# records = extract_videos_mdetr(aptm_sampled.iloc[:100], mp4_files, root_dir, delta = delta, step = step, frame_sample_rate = frame_sample_rate, num_images = num_images, model = yolo, bbox_conf = bbox_conf)
# records = extract_videos(aptm_sampled.iloc[:100], mp4_files, root_dir, delta = delta, step = step, frame_sample_rate = frame_sample_rate, num_images = num_images, model = yolo, bbox_conf = bbox_conf)
# records = extract_videos(aptm_sampled, mp4_files, root_dir, delta = delta, step = step, frame_sample_rate = frame_sample_rate, num_images = num_images, model = yolo, bbox_conf = bbox_conf)
# records = extract_videos(aptm_sampled.iloc[:10], mp4_files, root_dir, delta = delta, step = step)
# records = extract_videos(aptm_sampled, mp4_files, root_dir, delta = delta, step = step)

# End time
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

formatted_time = format_time(elapsed_time)
records[0].to_csv(root_dir + '/videos_metadata.csv', index=False)
records[1].to_csv(root_dir + '/images_metadata.csv', index=False)
records[2].to_csv(root_dir + '/indices_metadata.csv', index=False)
# records.to_csv('train_video_extracted_metadata.csv', index=False)

# Display the elapsed time
print(f"Time of execution: {formatted_time} from {elapsed_time}")

Processing items:   0%|                                                                          | 0/10 [00:00<?, ?it/s]

468.0 478.0 2022-02-18 19:00:01 2022-02-18 18:59:51 2022-02-18 19:07:49 2022-02-18 19:07:59 /data/public/heureux_2022-02-16_2022-04-16/2022-02-16-2022-02-21/N862A6_ch2_main_20220218190001_20220218200001.mp4 6791

0: 384x640 (no detections), 0.2ms
1: 384x640 (no detections), 0.2ms
2: 384x640 (no detections), 0.2ms
3: 384x640 (no detections), 0.2ms
4: 384x640 (no detections), 0.2ms
5: 384x640 (no detections), 0.2ms
6: 384x640 (no detections), 0.2ms
7: 384x640 (no detections), 0.2ms
8: 384x640 (no detections), 0.2ms
9: 384x640 (no detections), 0.2ms
10: 384x640 (no detections), 0.2ms
11: 384x640 (no detections), 0.2ms
12: 384x640 (no detections), 0.2ms
13: 384x640 (no detections), 0.2ms
14: 384x640 (no detections), 0.2ms
15: 384x640 (no detections), 0.2ms
16: 384x640 (no detections), 0.2ms
17: 384x640 (no detections), 0.2ms
18: 384x640 (no detections), 0.2ms
19: 384x640 (no detections), 0.2ms
20: 384x640 (no detections), 0.2ms
21: 384x640 (no detections), 0.2ms
22: 384x640 (no detections)

Processing items:  10%|██████▌                                                           | 1/10 [00:04<00:37,  4.14s/it]

3265.0 3275.0 2022-02-18 18:00:01 2022-02-18 18:52:10 2022-02-18 18:54:26 2022-02-18 18:54:36 /data/public/heureux_2022-02-16_2022-04-16/2022-02-16-2022-02-21/N862A6_ch2_main_20220218180001_20220218190001.mp4 6768

0: 384x640 1 Face, 0.2ms
1: 384x640 1 Face, 0.2ms
2: 384x640 (no detections), 0.2ms
3: 384x640 (no detections), 0.2ms
4: 384x640 (no detections), 0.2ms
5: 384x640 (no detections), 0.2ms
6: 384x640 (no detections), 0.2ms
7: 384x640 (no detections), 0.2ms
8: 384x640 (no detections), 0.2ms
9: 384x640 (no detections), 0.2ms
10: 384x640 (no detections), 0.2ms
11: 384x640 (no detections), 0.2ms
12: 384x640 (no detections), 0.2ms
13: 384x640 (no detections), 0.2ms
14: 384x640 (no detections), 0.2ms
15: 384x640 (no detections), 0.2ms
16: 384x640 (no detections), 0.2ms
17: 384x640 (no detections), 0.2ms
18: 384x640 (no detections), 0.2ms
19: 384x640 (no detections), 0.2ms
20: 384x640 (no detections), 0.2ms
21: 384x640 (no detections), 0.2ms
22: 384x640 (no detections), 0.2ms
23: 384x

Processing items:  20%|█████████████▏                                                    | 2/10 [00:12<00:51,  6.42s/it]

...

2453.0 2463.0 2022-02-18 17:00:08 2022-02-18 17:40:38 2022-02-18 17:41:01 2022-02-18 17:41:11 /data/public/heureux_2022-02-16_2022-04-16/2022-02-16-2022-02-21/N862A6_ch9_main_20220218170008_20220218180008.mp4 6353

0: 384x640 (no detections), 0.2ms
1: 384x640 (no detections), 0.2ms
2: 384x640 (no detections), 0.2ms
3: 384x640 (no detections), 0.2ms
4: 384x640 (no detections), 0.2ms
5: 384x640 (no detections), 0.2ms
6: 384x640 (no detections), 0.2ms
7: 384x640 (no detections), 0.2ms
8: 384x640 (no detections), 0.2ms
9: 384x640 (no detections), 0.2ms
10: 384x640 (no detections), 0.2ms
11: 384x640 (no detections), 0.2ms
12: 384x640 (no detections), 0.2ms
13: 384x640 (no detections), 0.2ms
14: 384x640 (no detections), 0.2ms
15: 384x640 (no detections), 0.2ms
16: 384x640 (no detections), 0.2ms
17: 384x640 (no detections), 0.2ms
18: 384x640 (no detections), 0.2ms
19: 384x640 (no detections), 0.2ms
20: 384x640 (no detections), 0.2ms
21: 384x640 (no detections), 0.2ms
22: 384x640 (no detection

Processing items:  90%|███████████████████████████████████████████████████████████▍      | 9/10 [00:47<00:05,  5.54s/it]

2309.0 2319.0 2022-02-18 17:00:01 2022-02-18 17:32:07 2022-02-18 17:38:30 2022-02-18 17:38:40 /data/public/heureux_2022-02-16_2022-04-16/2022-02-16-2022-02-21/N862A6_ch2_main_20220218170001_20220218180001.mp4 6773

0: 384x640 (no detections), 0.2ms
1: 384x640 (no detections), 0.2ms
2: 384x640 (no detections), 0.2ms
3: 384x640 (no detections), 0.2ms
4: 384x640 (no detections), 0.2ms
5: 384x640 (no detections), 0.2ms
6: 384x640 (no detections), 0.2ms
7: 384x640 (no detections), 0.2ms
8: 384x640 (no detections), 0.2ms
9: 384x640 (no detections), 0.2ms
10: 384x640 (no detections), 0.2ms
11: 384x640 (no detections), 0.2ms
12: 384x640 (no detections), 0.2ms
13: 384x640 (no detections), 0.2ms
14: 384x640 (no detections), 0.2ms
15: 384x640 (no detections), 0.2ms
16: 384x640 (no detections), 0.2ms
17: 384x640 (no detections), 0.2ms
18: 384x640 (no detections), 0.2ms
19: 384x640 (no detections), 0.2ms
20: 384x640 (no detections), 0.2ms
21: 384x640 (no detections), 0.2ms
22: 384x640 (no detection

Processing items: 100%|█████████████████████████████████████████████████████████████████| 10/10 [00:51<00:00,  5.15s/it]

Time of execution: 00:00:51 from 51.51137375831604
